In [1]:
import warnings
warnings.filterwarnings('ignore')

# Matplotlib inline magic command
%matplotlib inline

In [2]:
# Initial imports
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from collections import Counter
import psycopg2
from config import db_password
import json
from pandas.io.json import json_normalize
import mysql.connector
from sqlalchemy import create_engine
from os import walk
import time
from pprint import pprint

In [3]:
# Scikit-Learn dependencies
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, classification, report
from sklearn.feature_selection import RFE

In [4]:
# Connect to AWS
def getDBengine1():
    user = 'postgres'
    passw = db_password
    host = 'policekillings.cv95lgysyvwq.us-east-1.rds.amazonaws.com'
    port = 5432
    database = 'PoliceKillings'
    engine_1 = create_engine(f'postgres+psycopg2://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database ,  echo=False)
    return engine_1

In [5]:
# Load database into DataFrame
def getData1():
    s_statement = "select * from public.police_killings"
    # df = pd.read_sql(s_statement, con=getDBengine())
    try:
        df = pd.read_sql(s_statement, con=getDBengine1())
    except:
        print('failed')
    return df

In [6]:
killings_df = getData1()
killings_df.shape
print(killings_df.columns)

Index(['Victim_Age', 'Victim_Gender', 'Victim_Race', 'Date', 'City', 'State',
       'County', 'Responsible_Agency', 'Cause_of_death', 'Brief_Description ',
       'Criminal_Charges', 'Mental_Illness', 'Armed_Status', 'Threat_Level ',
       'Fleeing ', 'Body_Camera', 'Geography', 'Encounter_Type ',
       'Initial_Reason_for_Encounter ', 'Call_for_Service'],
      dtype='object')


In [9]:
# Delete columns
del killings_df['Victim_Gender']
#del killings_df['City']
#del killings_df['State']
#del killings_df['Criminal_Charges']
#del killings_df['Brief_Description']

In [10]:
# Find null values
for column in killings_df.columns:
    print(f'Column {column} has {killings_df[column].isnull().sum()} null values')

Column Victim_Age has 8 null values
Column Victim_Race has 0 null values
Column Date has 0 null values
Column County has 26 null values
Column Responsible_Agency has 4 null values
Column Cause_of_Death has 0 null values
Column Mental_Illness has 69 null values
Column Armed_Status has 6 null values
Column Threat_Level has 2394 null values
Column Fleeing has 3077 null values
Column Body_Camera has 3147 null values
Column Geography has 31 null values
Column Encounter_Type has 4310 null values
Column Initial_Reason_for_Encounter has 4310 null values
Column Call_for_Service has 5539 null values


In [11]:
# Replace empty/blank values for Age, Threat_Level, Fleeing, Body_Camera, Encounter_Type, Initial_Reason_for_Encounter, 
# Call_for_Service and re-verify null values

# Column: Age
killings_df['Victim_Age'].replace({'Unknown': '0'}, inplace = True) 
                                                                    
# Column: Threat_Level
killings_df['Threat_Level'].replace({None: 'undetermined'}, inplace = True)

# Column: Fleeing
killings_df['Fleeing'].replace({None: 'unknown'}, inplace = True)

# Column: Body_Camera
killings_df['Body_Camera'].replace({None: 'unknown'}, inplace = True)

# Column: Encounter_Type
killings_df['Encounter_Type'].replace({None: 'Unknown'}, inplace = True)

# Column: Initial_Reason_for_Encounter
killings_df['Initial_Reason_for_Encounter'].replace({None: 'unknown'}, inplace = True)

# Column: Call_for_Service
killings_df['Call_for_Service'].replace({None: 'Unavailable'}, inplace = True)

# Verify null values
for column in killings_df.columns:
    print(f'Column {column} has {killings_df[column].isnull().sum()} null values')

Column Victim_Age has 8 null values
Column Victim_Race has 0 null values
Column Date has 0 null values
Column County has 26 null values
Column Responsible_Agency has 4 null values
Column Cause_of_Death has 0 null values
Column Mental_Illness has 69 null values
Column Armed_Status has 6 null values
Column Threat_Level has 0 null values
Column Fleeing has 0 null values
Column Body_Camera has 0 null values
Column Geography has 31 null values
Column Encounter_Type has 0 null values
Column Initial_Reason_for_Encounter has 0 null values
Column Call_for_Service has 0 null values


In [12]:
# Drop null values
killings_df = killings_df.dropna(how = 'any')
killings_df = killings_df.reset_index().drop(['index'], axis = 1)
killings_df

,Victim_Age,Victim_Race,Date,County,Responsible_Agency,Cause_of_Death,Mental_Illness,Armed_Status,Threat_Level,Fleeing,Body_Camera,Geography,Encounter_Type,Initial_Reason_for_Encounter,Call_for_Service
0,46,Hispanic,2021-04-14,Harris,Harris County Sheriff's Office,"Gunshot, Taser",Yes,Allegedly Armed,other,unknown,no,Suburban,Mental Health/Welfare Check,mental health crisis,Yes
1,48,Unknown Race,2021-04-10,Los Angeles,San Fernando Police Department,Gunshot,No,Allegedly Armed,attack,not fleeing,no,Urban,Part 1 Violent Crime,gunshots,Yes
2,0,Unknown Race,2021-04-10,Winnebago,Winnebago County Sheriff's Office,Gunshot,No,Unclear,undetermined,unknown,no,Suburban,Domestic disturbance,domestic disturbance,No
3,0,Unknown Race,2021-04-10,Jefferson,Jefferson Parish Sheriff's Office,Gunshot,No,Allegedly Armed,attack,unknown,no,Urban,Domestic disturbance,domestic disturbance,Yes
4,50,Unknown Race,2021-04-09,Clark,U.S. Marshals Service,Gunshot,No,Allegedly Armed,attack,not fleeing,no,Suburban,Other Crimes Against People,warrant (threatening judge),No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,21,White,2013-01-01,Douglas,Douglas County Sheriff's Office,Gunshot,Drug or alcohol use,Allegedly Armed,undetermined,unknown,unknown,Rural,Unknown,unknown,Unavailable
8970,26,White,2013-01-01,Blue Earth,Mankato Department of Public Safety,Taser,No,Unarmed/Did Not Have Actual Weapon,undetermined,unknown,unknown,Suburban,Unknown,unknown,Unavailable
8971,21,Hispanic,2013-01-01,Pueblo,Pueblo Police Department,Gunshot,No,Allegedly Armed,undetermined,unknown,unknown,Suburban,Unknown,unknown,Unavailable
8972,49,Hispanic,2013-01-01,San Juan,Farmington Police Department,Gunshot,No,Allegedly Armed,undetermined,unknown,unknown,Urban,Unknown,unknown,Unavailable


In [13]:
# Inspect column data types
killings_df.dtypes

Victim_Age                              object
Victim_Race                             object
Date                            datetime64[ns]
County                                  object
Responsible_Agency                      object
Cause_of_Death                          object
Mental_Illness                          object
Armed_Status                            object
Threat_Level                            object
Fleeing                                 object
Body_Camera                             object
Geography                               object
Encounter_Type                          object
Initial_Reason_for_Encounter            object
Call_for_Service                        object
dtype: object

In [14]:
# Change Victim_Age data type from object to int
killings_df['Victim_Age'] = killings_df['Victim_Age'].astype(str).astype(int)
killings_df.dtypes

Victim_Age                               int32
Victim_Race                             object
Date                            datetime64[ns]
County                                  object
Responsible_Agency                      object
Cause_of_Death                          object
Mental_Illness                          object
Armed_Status                            object
Threat_Level                            object
Fleeing                                 object
Body_Camera                             object
Geography                               object
Encounter_Type                          object
Initial_Reason_for_Encounter            object
Call_for_Service                        object
dtype: object

In [15]:
# Extract year from Date and create new column Year
killings_df['Year'] = pd.DatetimeIndex(killings_df['Date']).year

# Extract month from Date and create new column Month
killings_df['Month'] = pd.DatetimeIndex(killings_df['Date']).month

# Extract day from Date and create new column Day
killings_df['Day'] = pd.DatetimeIndex(killings_df['Date']).day

# Get day of week from Date and create new column Day_of_Week
killings_df['Day_of_Week'] = killings_df['Date'].dt.day_name()

In [16]:
# Get holiday from Date and create new column Holiday
dr = pd.date_range(start='2013-01-01', end='2021-04-18')
df = pd.DataFrame()
df['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

killings_df['Holiday'] = killings_df['Date'].dt.date.astype('datetime64').isin(holidays)

In [ ]:
#killings_df

In [17]:
# Use get_dummies() to transform text values into numerical values
killings_encoded = pd.get_dummies(killings_df, columns = 
                                  ['Cause_of_Death', 'Mental_Illness', 'Armed_Status', 'Threat_Level', 'Fleeing', 
                                   'Body_Camera', 'Geography', 'County', 'Responsible_Agency', 'Encounter_Type', 
                                   'Initial_Reason_for_Encounter', 'Call_for_Service'])

killings_encoded.head()

,Victim_Age,Victim_Race,Date,Year,Month,Day,Day_of_Week,Holiday,Cause_of_Death_Asphyxiated,"Cause_of_Death_Baton, Pepper Spray, Physical Restraint",...,"Initial_Reason_for_Encounter_wellness check/erratic behavior, other: danger to self (suicide)","Initial_Reason_for_Encounter_wellness check/erratic behavior, shooting/shots fired","Initial_Reason_for_Encounter_wellness check/erratic behavior, warrant (failure to appear)","Initial_Reason_for_Encounter_wellness check/erratic behavior, weapons offense","Initial_Reason_for_Encounter_wellness check/erratic behavior, weapons offense (gun)",Call_for_Service_No,Call_for_Service_No,Call_for_Service_Unavailable,Call_for_Service_Yes,Call_for_Service_yes
0,46,Hispanic,2021-04-14,2021,4,14,Wednesday,False,0,0,...,0,0,0,0,0,0,0,0,1,0
1,48,Unknown Race,2021-04-10,2021,4,10,Saturday,False,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,Unknown Race,2021-04-10,2021,4,10,Saturday,False,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,Unknown Race,2021-04-10,2021,4,10,Saturday,False,0,0,...,0,0,0,0,0,0,0,0,1,0
4,50,Unknown Race,2021-04-09,2021,4,9,Friday,False,0,0,...,0,0,0,0,0,1,0,0,0,0


In [18]:
# Drop Date column
killings_df = killings_df.drop(['Date'], axis = 1)

In [19]:
killings_df['Victim_Race'].value_counts()

White               3935
Black               2259
Hispanic            1566
Unknown Race         903
Asian                134
Native American      127
Pacific Islander      50
Name: Victim_Race, dtype: int64

In [21]:
# Convert Victim_Race values to numbers
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('White', 1)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Black', 2)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Hispanic', 3)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Unknown Race', 4)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Asian', 5)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Native American', 6)
killings_df['Victim_Race'] = killings_df['Victim_Race'].replace('Pacific Islander', 7)

killings_df['Victim_Race']

0       3
1       4
2       4
3       4
4       4
       ..
8969    1
8970    1
8971    3
8972    3
8973    2
Name: Victim_Race, Length: 8974, dtype: int64

## Split Data into Training and Testing Sets

In [22]:
# Create features
X = pd.get_dummies(killings_df.drop(columns = ['Victim_Race']))

# Create target
y = pd.DataFrame(killings_df['Victim_Race'])

In [23]:
X.describe()

,Victim_Age,Year,Month,Day,County_Acadia,County_Acadiana,County_Ada,County_Adair,County_Adams,County_Aiken,...,Call_for_Service_Unavailable,Call_for_Service_Yes,Call_for_Service_yes,Day_of_Week_Friday,Day_of_Week_Monday,Day_of_Week_Saturday,Day_of_Week_Sunday,Day_of_Week_Thursday,Day_of_Week_Tuesday,Day_of_Week_Wednesday
count,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.00000,...,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000,8974.000000
mean,35.541899,2016.653109,6.298418,15.607421,0.000334,0.000111,0.001449,0.000223,0.005014,0.00078,...,0.636617,0.239692,0.000223,0.146757,0.135614,0.134834,0.137508,0.148540,0.149766,0.146980
std,14.557785,2.380105,3.458029,8.784326,0.018282,0.010556,0.038035,0.014928,0.070639,0.02792,...,0.481001,0.426920,0.014928,0.353884,0.342397,0.341565,0.344402,0.355654,0.356862,0.354106
min,0.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.000000,2015.000000,3.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,34.000000,2017.000000,6.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,45.000000,2019.000000,9.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,107.000000,2021.000000,12.000000,31.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# Check balance of target values
y['Victim_Race'].value_counts()

1    3935
2    2259
3    1566
4     903
5     134
6     127
7      50
Name: Victim_Race, dtype: int64

In [25]:
# Train, test, and split model
X_train, X_test, y_train, y_test = train_test_split(X,y)
                                                                                                   
y_train.shape

(6730, 1)

## Random Forest Model

In [26]:
# Create a random forest classifier
rf_model = RandomForestClassifier()
rf_model = rf_model.fit(X_train, y_train)

In [27]:
def multiclass_roc_auc_score(truth, pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(truth)
    truth = lb.transform(truth)
    pred = lb.transform(pred)
    return roc_auc_score(truth, pred, average=average)

In [29]:
y_test_np = np.array(y_test)
rf_y_pred_np = np.array(rf_y_pred)

In [30]:
y_test_np = y_test_np.reshape(-1, 1)
rf_y_pred_np = rf_y_pred_np.reshape(-1, 1)

In [31]:
# Calculate ROC accuracy score
multiclass_roc_auc_score(y_test, rf_y_pred)

0.5651455999556888

In [32]:
# Print classification report
print(classification_report(y_test, rf_y_pred))

              precision    recall  f1-score   support

           1       0.52      0.88      0.65       973
           2       0.56      0.41      0.47       579
           3       0.59      0.21      0.31       382
           4       0.41      0.07      0.12       234
           5       1.00      0.03      0.05        38
           6       0.00      0.00      0.00        27
           7       0.33      0.09      0.14        11

    accuracy                           0.53      2244
   macro avg       0.49      0.24      0.25      2244
weighted avg       0.53      0.53      0.47      2244



In [ ]:
# List feature sorted in descending order by feature importance - is this needed?
sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)

In [33]:
# Random Forest model prediction
print(rf_y_pred)

[1 1 1 ... 1 2 1]


In [61]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [2],
    'max_leaf_nodes': [5],
    'n_estimators': [200],
    'oob_score': [True],
    'random_state': [0],
    'max_samples': [10],
    'max_features': [10],
    'min_samples_split': [50]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid, 
                      cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)
# Fit the grid search to the data
grid_search.fit(X_train, y_train);
grid_search.best_params_
best_grid = grid_search.best_estimator_
pprint(best_grid.get_params())
selector = RFE(rf_model, step=1, verbose=3)
selector = selector.fit(X_train, y_train)
print("Features sorted by their rank:")
pprint(sorted(zip(map(lambda x: round(x, 4), selector.ranking_), X)))

## Decision Tree Model

In [35]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifier
clf = clf.fit(X_train, y_train)

# Predict response for test dataset
clf_y_pred = clf.predict(X_test)

In [36]:
def multiclass_roc_auc_score(truth, pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(truth)
    truth = lb.transform(truth)
    pred = lb.transform(pred)
    return roc_auc_score(truth, pred, average=average)

In [42]:
y_test_np = np.array(y_test)
clf_y_pred_np = np.array(clf_y_pred)

In [43]:
y_test_np = y_test_np.reshape(-1, 1)
clf_y_pred_np = clf_y_pred_np.reshape(-1, 1)

In [62]:
# Calculate ROC accuracy score
multiclass_roc_auc_score(y_test, clf_y_pred)

0.5608518576980569

In [45]:
# Print classification report
print(classification_report(y_test, clf_y_pred))

              precision    recall  f1-score   support

           1       0.52      0.88      0.65       973
           2       0.56      0.41      0.47       579
           3       0.59      0.21      0.31       382
           4       0.41      0.07      0.12       234
           5       1.00      0.03      0.05        38
           6       0.00      0.00      0.00        27
           7       0.33      0.09      0.14        11

    accuracy                           0.53      2244
   macro avg       0.49      0.24      0.25      2244
weighted avg       0.53      0.53      0.47      2244



In [46]:
# Decision Tree model prediction
print(clf_y_pred)

[1 1 1 ... 1 2 1]


In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [2],
    'max_leaf_nodes': [5],
    'n_estimators': [200],
    'oob_score': [True],
    'random_state': [0],
    'max_samples': [10],
    'max_features': [10],
    'min_samples_split': [50]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid, 
                      cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)
# Fit the grid search to the data
grid_search.fit(X_train, y_train);
grid_search.best_params_
best_grid = grid_search.best_estimator_
pprint(best_grid.get_params())
selector = RFE(rf_model, step=1, verbose=3)
selector = selector.fit(X_train, y_train)
print("Features sorted by their rank:")
pprint(sorted(zip(map(lambda x: round(x, 4), selector.ranking_), X)))

## Logistic Regression Model

In [52]:
# Create logistic regression instance
logreg = LogisticRegression()

# Fit the model
logreg.fit(X_train, y_train)

# Predict response for test dataset
logreg_y_pred = logreg.predict(X_test)

In [53]:
def multiclass_roc_auc_score(truth, pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(truth)
    truth = lb.transform(truth)
    pred = lb.transform(pred)
    return roc_auc_score(truth, pred, average=average)

In [55]:
y_test_np = np.array(y_test)
logreg_y_pred_np = np.array(logreg_y_pred)

In [56]:
y_test_np = y_test_np.reshape(-1, 1)
logreg_y_pred_np = logreg_y_pred_np.reshape(-1, 1)

In [63]:
# Calculate ROC accuracy score
multiclass_roc_auc_score(y_test, logreg_y_pred)

0.5161113606426945

In [57]:
# Print classification report
print(classification_report(y_test, logreg_y_pred))

              precision    recall  f1-score   support

           1       0.47      0.94      0.63       973
           2       0.38      0.20      0.26       579
           3       0.00      0.00      0.00       382
           4       0.00      0.00      0.00       234
           5       0.00      0.00      0.00        38
           6       0.00      0.00      0.00        27
           7       0.00      0.00      0.00        11

    accuracy                           0.46      2244
   macro avg       0.12      0.16      0.13      2244
weighted avg       0.30      0.46      0.34      2244



In [58]:
# Logistic Regression model prediction
print(y_pred)

[1 1 1 ... 1 1 1]


In [59]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [2],
    'max_leaf_nodes': [5],
    'n_estimators': [200],
    'oob_score': [True],
    'random_state': [0],
    'max_samples': [10],
    'max_features': [10],
    'min_samples_split': [50]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid, 
                      cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)
# Fit the grid search to the data
grid_search.fit(X_train, y_train);
grid_search.best_params_
best_grid = grid_search.best_estimator_
pprint(best_grid.get_params())
selector = RFE(rf_model, step=1, verbose=3)
selector = selector.fit(X_train, y_train)
print("Features sorted by their rank:")
pprint(sorted(zip(map(lambda x: round(x, 4), selector.ranking_), X)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 2,
 'max_features': 10,
 'max_leaf_nodes': 5,
 'max_samples': 10,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 50,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}
Fitting estimator with 5996 features.
Fitting estimator with 5995 features.
Fitting estimator with 5994 features.
Fitting estimator with 5993 features.
Fitting estimator with 5992 features.
Fitting estimator with 5991 features.
Fitting estimator with 5990 features.
Fitting estimator with 5989 features.
Fitting estimator with 5988 features.
Fitting estimator with 5987 features.
Fitting estimator with 5986 features.
Fitting estimator with 5985 features.
Fitting estimator with 5984 features.
Fitting estimator with 

Fitting estimator with 5793 features.
Fitting estimator with 5792 features.
Fitting estimator with 5791 features.
Fitting estimator with 5790 features.
Fitting estimator with 5789 features.
Fitting estimator with 5788 features.
Fitting estimator with 5787 features.
Fitting estimator with 5786 features.
Fitting estimator with 5785 features.
Fitting estimator with 5784 features.
Fitting estimator with 5783 features.
Fitting estimator with 5782 features.
Fitting estimator with 5781 features.
Fitting estimator with 5780 features.
Fitting estimator with 5779 features.
Fitting estimator with 5778 features.
Fitting estimator with 5777 features.
Fitting estimator with 5776 features.
Fitting estimator with 5775 features.
Fitting estimator with 5774 features.
Fitting estimator with 5773 features.
Fitting estimator with 5772 features.
Fitting estimator with 5771 features.
Fitting estimator with 5770 features.
Fitting estimator with 5769 features.
Fitting estimator with 5768 features.
Fitting esti

Fitting estimator with 5577 features.
Fitting estimator with 5576 features.
Fitting estimator with 5575 features.
Fitting estimator with 5574 features.
Fitting estimator with 5573 features.
Fitting estimator with 5572 features.
Fitting estimator with 5571 features.
Fitting estimator with 5570 features.
Fitting estimator with 5569 features.
Fitting estimator with 5568 features.
Fitting estimator with 5567 features.
Fitting estimator with 5566 features.
Fitting estimator with 5565 features.
Fitting estimator with 5564 features.
Fitting estimator with 5563 features.
Fitting estimator with 5562 features.
Fitting estimator with 5561 features.
Fitting estimator with 5560 features.
Fitting estimator with 5559 features.
Fitting estimator with 5558 features.
Fitting estimator with 5557 features.
Fitting estimator with 5556 features.
Fitting estimator with 5555 features.
Fitting estimator with 5554 features.
Fitting estimator with 5553 features.
Fitting estimator with 5552 features.
Fitting esti

Fitting estimator with 5361 features.
Fitting estimator with 5360 features.
Fitting estimator with 5359 features.
Fitting estimator with 5358 features.
Fitting estimator with 5357 features.
Fitting estimator with 5356 features.
Fitting estimator with 5355 features.
Fitting estimator with 5354 features.
Fitting estimator with 5353 features.
Fitting estimator with 5352 features.
Fitting estimator with 5351 features.
Fitting estimator with 5350 features.
Fitting estimator with 5349 features.
Fitting estimator with 5348 features.
Fitting estimator with 5347 features.
Fitting estimator with 5346 features.
Fitting estimator with 5345 features.
Fitting estimator with 5344 features.
Fitting estimator with 5343 features.
Fitting estimator with 5342 features.
Fitting estimator with 5341 features.
Fitting estimator with 5340 features.
Fitting estimator with 5339 features.
Fitting estimator with 5338 features.
Fitting estimator with 5337 features.
Fitting estimator with 5336 features.
Fitting esti

Fitting estimator with 5145 features.
Fitting estimator with 5144 features.
Fitting estimator with 5143 features.
Fitting estimator with 5142 features.
Fitting estimator with 5141 features.
Fitting estimator with 5140 features.
Fitting estimator with 5139 features.
Fitting estimator with 5138 features.
Fitting estimator with 5137 features.
Fitting estimator with 5136 features.
Fitting estimator with 5135 features.
Fitting estimator with 5134 features.
Fitting estimator with 5133 features.
Fitting estimator with 5132 features.
Fitting estimator with 5131 features.
Fitting estimator with 5130 features.
Fitting estimator with 5129 features.
Fitting estimator with 5128 features.
Fitting estimator with 5127 features.
Fitting estimator with 5126 features.
Fitting estimator with 5125 features.
Fitting estimator with 5124 features.
Fitting estimator with 5123 features.
Fitting estimator with 5122 features.
Fitting estimator with 5121 features.
Fitting estimator with 5120 features.
Fitting esti

...

Fitting estimator with 3417 features.
Fitting estimator with 3416 features.
Fitting estimator with 3415 features.
Fitting estimator with 3414 features.
Fitting estimator with 3413 features.
Fitting estimator with 3412 features.
Fitting estimator with 3411 features.
Fitting estimator with 3410 features.
Fitting estimator with 3409 features.
Fitting estimator with 3408 features.
Fitting estimator with 3407 features.
Fitting estimator with 3406 features.
Fitting estimator with 3405 features.
Fitting estimator with 3404 features.
Fitting estimator with 3403 features.
Fitting estimator with 3402 features.
Fitting estimator with 3401 features.
Fitting estimator with 3400 features.
Fitting estimator with 3399 features.
Fitting estimator with 3398 features.
Fitting estimator with 3397 features.
Fitting estimator with 3396 features.
Fitting estimator with 3395 features.
Fitting estimator with 3394 features.
Fitting estimator with 3393 features.
Fitting estimator with 3392 features.
Fitting esti

Fitting estimator with 3201 features.
Fitting estimator with 3200 features.
Fitting estimator with 3199 features.
Fitting estimator with 3198 features.
Fitting estimator with 3197 features.
Fitting estimator with 3196 features.
Fitting estimator with 3195 features.
Fitting estimator with 3194 features.
Fitting estimator with 3193 features.
Fitting estimator with 3192 features.
Fitting estimator with 3191 features.
Fitting estimator with 3190 features.
Fitting estimator with 3189 features.
Fitting estimator with 3188 features.
Fitting estimator with 3187 features.
Fitting estimator with 3186 features.
Fitting estimator with 3185 features.
Fitting estimator with 3184 features.
Fitting estimator with 3183 features.
Fitting estimator with 3182 features.
Fitting estimator with 3181 features.
Fitting estimator with 3180 features.
Fitting estimator with 3179 features.
Fitting estimator with 3178 features.
Fitting estimator with 3177 features.
Fitting estimator with 3176 features.
Fitting esti

  'Office'),
 (1, 'Responsible_Agency_Middletown Police Department'),
 (1, "Responsible_Agency_Midland County Sheriff's Office"),
 (1, 'Responsible_Agency_Midland Police Department'),
 (1, 'Responsible_Agency_Midlothian Police Department'),
 (1, 'Responsible_Agency_Midwest City Police Department'),
 (1, 'Responsible_Agency_Milford Police Department'),
 (1, 'Responsible_Agency_Millbrook Police Department'),
 (1, "Responsible_Agency_Mille Lacs County Sheriff's Office"),
 (1, 'Responsible_Agency_Milton Police Department'),
 (1, 'Responsible_Agency_Milwaukee Police Department'),
 (1, 'Responsible_Agency_Minneapolis Police Department'),
 (1, 'Responsible_Agency_Minnetonka Police Department'),
 (1, 'Responsible_Agency_Miramar Police Department'),
 (1, 'Responsible_Agency_Mission Police Department'),
 (1, 'Responsible_Agency_Mississippi Highway Patrol'),
 (1, 'Responsible_Agency_Missouri City Police Department'),
 (1, 'Responsible_Agency_Missouri State Highway Patrol'),
 (1,
  'Responsible_Ag

 (1208, 'County_Geary'),
 (1209,
  "Responsible_Agency_San Luis Obispo County Sheriff's Office, California "
  'Highway Patrol'),
 (1210, 'Initial_Reason_for_Encounter_Intervening in domestic dispute'),
 (1211,
  'Initial_Reason_for_Encounter_traffic stop - man leaving suspected drug '
  'house'),
 (1212, 'Responsible_Agency_Des Plaines Police Department'),
 (1213, 'Responsible_Agency_Fox Crossing Police Department'),
 (1214,
  "Responsible_Agency_Citrus County Sheriff's Office, Hernando County "
  "Sheriff's Office"),
 (1215, 'County_Labette'),
 (1216, 'Responsible_Agency_Wyoming Police Department'),
 (1217, "Responsible_Agency_Walton County Sheriff's Office"),
 (1218,
  'Responsible_Agency_Marion Police Department, Illinois State Police, '
  "Williamson County Sheriff's Department"),
 (1219, 'Responsible_Agency_Bourbon Police Department'),
 (1220,
  'Initial_Reason_for_Encounter_Conducting probation compliance check on '
  'someone else'),
 (1221, "Responsible_Agency_Wagoner County S

 (2932, "Responsible_Agency_Barren County Sheriff's Office"),
 (2933,
  'Responsible_Agency_Gwinnett County Police Department, Snellville Police '
  'Department'),
 (2934,
  "Responsible_Agency_Snohomish County Sheriff's Office, Monroe Police "
  'Department'),
 (2935, "Responsible_Agency_St. Clair County Sheriff's Office"),
 (2936,
  'Responsible_Agency_Green Township Police Department, Cheviot Police '
  'Department'),
 (2937, 'Cause_of_Death_Beaten/Bludgeoned with instrument'),
 (2938, "Responsible_Agency_Randall County Sheriff's Office"),
 (2939,
  "Responsible_Agency_Monroe County Sheriff's Office, Sparta Police "
  'Department'),
 (2940, 'Responsible_Agency_Liberty Police Department'),
 (2941,
  'Responsible_Agency_Edina Police Department, Richfield Police Department'),
 (2942, "Responsible_Agency_Wheeler County Sheriff's Office"),
 (2943, 'Initial_Reason_for_Encounter_mass shooting'),
 (2944,
  "Responsible_Agency_Scott County Sheriff's Office, Georgetown Police "
  'Department,